# ResponseGetData

> preferred response class for all API requests

In [ ]:
#| default_exp ResponseGetData

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass, field
from fastcore.utils import patch_to
from fastcore.test import test_eq

import requests
import asyncio
import aiohttp

In [ ]:
#| export
API_Response = any

@dataclass
class ResponseGetData:
    """preferred response class for all API Requests"""
    status: int
    response: API_Response
    is_success: bool
    auth: dict = field(default=None)

In [ ]:
rgd = ResponseGetData(status = 200, response = "test", is_success = True)
rgd

ResponseGetData(status=200, response='test', is_success=True, auth=None)

In [ ]:
#| hide
test_eq(rgd.is_success , True)

# Classmethods from Response objects by library

This code base supports two API request libraries, `requests.request` (synchronous) and `aiohttp.ClientRequest` (asynchronous) this can be extended as new libraries emerge with different performance characteristics.

In [ ]:
#| export
@patch_to(ResponseGetData, cls_method = True)
def _from_requests_response(cls, res : requests.Response #requests response object
                           ) -> ResponseGetData:
    """returns ResponseGetData"""
    
    #JSON responses
    if res.ok and "application/json" in res.headers.get("Content-Type", {}):
        return ResponseGetData(
            status = res.status_code, 
            response = res.json(),
            is_success=True
        )

    #default text responses
    elif res.ok:
        return ResponseGetData(
            status=res.status_code,
            response=response.text,
            is_success=True
        )
    
    # errors
    return ResponseGetData(
        status=res.status_code,
        response=res.reason,
        is_success=False
    )

In [ ]:
show_doc(ResponseGetData._from_requests_response)

---

### ResponseGetData._from_requests_response

>      ResponseGetData._from_requests_response (res:requests.models.Response)

returns ResponseGetData

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| res | Response | requests response object |
| **Returns** | **ResponseGetData** |  |

In [ ]:
# test _from_requests_response returns ResponseGetData class

test_url = 'http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita'

res = requests.request(method = 'GET', url = test_url)

test_res = ResponseGetData._from_requests_response(res)
test_res.__dict__.keys()

In [ ]:
#| hide
test_eq(isinstance( test_res, ResponseGetData), True)

In [ ]:
#| export
@patch_to(ResponseGetData, cls_method = True)
async def _from_aiohttp_response(cls, res : aiohttp.ClientResponse #requests response object
                           ) -> ResponseGetData:
    
    """async method returns ResponseGetData"""
    
    if res.ok and "application/json" in res.headers.get("Content-Type", {}) :
        try:
            return ResponseGetData(
                status=res.status, response=await res.json(), is_success=True
            )

        # handle if unable to decode json()
        except asyncio.TimeoutError as e:
            print(e)
            print("response included json, but defaulted to backup decode method")

            return ResponseGetData(
                status=res.status, response=await res.read(), is_success=True
            )

        # response is text
    elif res.ok:
        return ResponseGetData(
            status=res.status, response=await res.text(), is_success=True
        )

    # response is error
    else:
        return ResponseGetData(status=res.status, response=res.reason, is_success=False)

In [ ]:
show_doc(ResponseGetData._from_aiohttp_response)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domo_library/ResponseGetData.py#L28){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData._from_aiohttp_response

>      ResponseGetData._from_aiohttp_response
>                                              (res:aiohttp.client_reqrep.Client
>                                              Response)

async method returns ResponseGetData

In [ ]:
# test _from_aiohttp_response returns ResponseGetData class

test_url = 'http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita'

session = aiohttp.ClientSession()
res = await session.get(url = test_url)
await session.close()

test_res = await ResponseGetData._from_aiohttp_response(res)
test_res.__dict__.keys()

In [ ]:
#| hide
test_eq(isinstance( test_res, ResponseGetData), True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()